In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv(r"dataset\fashion-mnist_train.csv")
test_df = pd.read_csv(r"dataset\fashion-mnist_test.csv")

train_data = np.array(train_df, dtype='float32')
test_data = np.array(test_df,dtype='float32')

x_train = train_data[:,1:]/255
y_train = train_data[:,0]

x_test = test_data[:,1:]/255
y_test = test_data[:,0]

In [3]:
x_train,x_validate,y_train,y_validate = train_test_split(
    x_train, y_train, test_size=0.2, random_state=12345,
)

In [4]:
im_rows = 28
im_cols = 28
batch_size = 512
im_shape = (im_rows,im_cols,1)

x_train = x_train.reshape(x_train.shape[0],*im_shape)
x_test = x_test.reshape(x_test.shape[0],*im_shape)
x_validate = x_validate.reshape(x_validate.shape[0],*im_shape)

In [5]:
cnn_model = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', 
           input_shape=im_shape, kernel_initializer='he_normal', name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.25, name='Dropout-1'),
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
    Dropout(0.25, name='Dropout-2'),
    Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3'),
    Dropout(0.4, name='Dropout-3'),
    Conv2D(256, kernel_size=3, activation='relu', name='Conv2D-4'),
    Dropout(0.6, name='Dropout-4'),
    Flatten(name='flatten'),
    Dense(256, activation='relu', name='Dense'),
    Dropout(0.6, name='Dropout'),
    Dense(10, activation='softmax', name='Output')
])

In [6]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(
    log_dir=r'logs\{}'.format('cnn_2layer'),
    write_graph=True,
    write_grads=True,
    histogram_freq=1,
    write_images=True,
)

cnn_model.compile(
    loss ='sparse_categorical_crossentropy',
    optimizer = Adam(lr=0.001),
    metrics=['accuracy']
)

In [7]:
cnn_model.fit(
    x_train, y_train, batch_size=batch_size,
    epochs=75, verbose=1,
    validation_data=(x_validate,y_validate),
    callbacks=[tensorboard]
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/75
48000/48000 [==============================] - 137s 3ms/step - loss: 1.0642 - acc: 0.6088 - val_loss: 0.5427 - val_acc: 0.7872
Epoch 2/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.5935 - acc: 0.7784 - val_loss: 0.4464 - val_acc: 0.8293
Epoch 3/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.5075 - acc: 0.8125 - val_loss: 0.3840 - val_acc: 0.8568
Epoch 4/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.4557 - acc: 0.8337 - val_loss: 0.3425 - val_acc: 0.8777
Epoch 5/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.4177 - acc: 0.8476 - val_loss: 0.3149 - val_acc: 0.8849
Epoch 6/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.3858 - acc: 0.8612 - val_loss: 0.2995 - val_acc: 0.8896
Epoch 7/75
48000/48000 [==============================] - 137s 3ms/step - loss: 0.3687 - acc: 0.8669 - val_loss: 0.2837 - 

In [8]:
score = cnn_model.evaluate(x_test,y_test,verbose=0)

print('test loss : {:4f}'.format(score[0]))
print('test score : {:4f}'.format(score[1]))

test loss : 0.178582
test score : 0.939600
